<a href="https://colab.research.google.com/github/kellianneyang/project-exploration/blob/main/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grades Project: Data Cleaning

The goal of this notebook is to prepare data for machine learning preprocessing. We would like to understand the data and understand what we want from the data.

This notebook will:
- Create new columns as necessary (to create new feature variables)
- Delete columns as necessary (to leave out data with high collinearity or high cardinality)
- Separate the data into three dataframes, each with its own target variable (there are multiple possible target variables that we are interested in, and will be applying our machine learning algorithms to them separately)
- Delete any rows from the dataframes that do not have values for the target variable

This notebook will NOT:
- Impute any missing values (this will be done in machine learning preprocessing)
- Delete any duplicated rows (")

## Overview


To use the data provided in this dataset (https://www.kaggle.com/datasets/Madgrades/uw-madison-courses), which is provided in multiple .csv files, for a machine learning model, we will need to merge the files to create one .csv file.

## .csv files and columns

Included in dataset as separate .csv files:
- schedules.csv: each row is a unique potential schedule
    - (schedule) uuid: unique identifier of schedules
    - start_time: start of class, in minutes
    - end_time: end of class, in minutes (drop -- high co-linearity with start_time)
    - mon: boolean, if class meets on monday
    - tues: "
    - wed: "
    - thurs: "
    - fri: "
    - sat: "
    - sun: "
      - sat and sun have very few values; combine to get more observations
- subjects.csv: each row is a unique subject
    - (subject) code: 3-digit unique identifier of subjects
    - (subject) name: name of subject
    - (subject) abbreviation: abbreviation (e.g. ENGL for English)
- teachings.csv: each row is a unique instructor
    - instructor_id: numeric unique identifier of instructor
    - section_uuid: section taught by instructor
- subject_memberships.csv: each row is a course offering (course offered in certain term; does not encompass all sections in that term)
    - subject_code: subject code associated with course offering
    - course_offering_uuid: unique identifier of course offerings
- sections.csv: each row is a section (specific instance of course at certain time in certain place in certain term)
    - (section) uuid: unique identifier of section (alphanumeric)
      - could have multiple section uuids for cross-listed sections
    - course_offering_uuid: unique identifier of course offering (course offered in certain term but encompasses all sections in that term) 
    - section_type: 3-letter identifier (e.g., LEC for lecture)
    - (section) number: 1-3-number of section (e.g., 301 for section 301)
    - room_uuid: unique identifier of room and building where section is held (including online and off-campus designations)
    - schedule_uuid: unique identifier of schedule for section
- rooms.csv: each row is a specific place
    - (room) uuid: unique identifier of building and room (inclduing off-campus and online)
    - facility_code: unique identifier of building
    - room_code: number of room within a building
- instructors.csv (will not use -- is essentially a duplicate of teachings; can use to look up instructors' names): each row is an instructor
    - (instructor) id: unique identifier of instructor
    - (instructor) name: instructor's name
- grade_distributions.csv: each row is a section's grades
    - course_offering_uuid: identifies the course offering (certain course in certain term, but not broken down into sections)
    - section_number: number of section for grades
    - a_count: number of As
    - ab_count: number of ABs (can be combined with As as needed)
    - b_count: number of Bs
    - bc_count: number of BCs (can be combined with Bs as needed)
    - c_count: number of Cs
    - d_count: number of Ds
    - f_count: number of Fs
    - s_count: satisfactory
    - u_count: unstatisfactory
    - cr_count: credit
    - n_count: no credit
    - p_count
    - i_count: incomplete
    - nw_count: no work
    - nr_count
- course_offerings.csv: each row is a course offering (course offered in certain term)
    - (course offering) uuid: unique identifier of course offering
    - course_uuid: course that the course offering belongs to
    - term_code: academic term when course offering was held
    - (course offering) name: name associate with course offering (can be different than course name)
- courses.csv: each row is a course (abstract; not associated with specific course offering)
    - (course) uuid: uniquely identifies the course
    - (course) name: name of course
    - (course) number: number in course catalog (e.g. 101 for ENGL 101)

## Feature variables

This dataset is large and has many variables we can choose from as our feature variables.

Many of the variables have a large number of unique values (high-cardinality), so we may have to drop those for our machine learning algorithm to be able to predict some variance.

We will also be combining some columns and dropping some columns for various reasons. 

## Target variables

We are interested in whether the feature variables can predict variation in grades. There are many grade types given (see source: https://guide.wisc.edu/undergraduate/#enrollmentandrecordstext), but to keep the problem manageable, we will narrow down the target grades.

We will instantiate three target variables and see if machine learning algorithms can predict the variation in any of them.

1. Proportion of A grades given (of all letter grades given)
2. Proportion of F grades given (of all letter grades given)
3. Average letter grade given (letter grades: A, AB, B, BC, C, D, and F)

# Preliminary Steps

In [ ]:
# import libraries
import pandas as pd
import seaborn as sns

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# load data
path = '/content/drive/MyDrive/Coding Dojo/Project 2/all_grades_data.csv'
df = pd.read_csv(path, low_memory = False)

In [ ]:
# inspect
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162710 entries, 0 to 162709
Data columns (total 44 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Unnamed: 0            162710 non-null  int64  
 1   section_uuid          162710 non-null  object 
 2   course_offering_uuid  162710 non-null  object 
 3   section_type          162710 non-null  object 
 4   section_number        162710 non-null  float64
 5   room_uuid             66778 non-null   object 
 6   schedule_uuid         162710 non-null  object 
 7   instructor_id         162682 non-null  float64
 8   facility_code         66778 non-null   object 
 9   room_code             64871 non-null   object 
 10  start_time            162710 non-null  float64
 11  end_time              162710 non-null  float64
 12  mon                   162710 non-null  bool   
 13  tues                  162710 non-null  bool   
 14  wed                   162710 non-null  bool   
 15  

In [ ]:
# inspect
df.head(10)

,Unnamed: 0,section_uuid,course_offering_uuid,section_type,section_number,room_uuid,schedule_uuid,instructor_id,facility_code,room_code,...,f_count,s_count,u_count,cr_count,n_count,p_count,i_count,nw_count,nr_count,other_count
0,0,214c946a-f56b-365b-a144-1d16049399ed,aa84af10-b176-3fdc-806b-ad7e4d45c93f,fld,1.0,NaN,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,1600463.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,1,5dffed06-3e1b-309a-92c9-4b755e20a7e8,a153948c-6fec-3de3-9a6d-3a7210340815,fld,10.0,NaN,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,693634.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,2,e3718659-dd65-30d3-987c-6b4092e85d72,a153948c-6fec-3de3-9a6d-3a7210340815,fld,13.0,NaN,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,590599.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,3,24085c2b-1756-320d-971a-a8cf64a78db0,a153948c-6fec-3de3-9a6d-3a7210340815,fld,14.0,NaN,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,664097.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,4,2187b6fd-09fb-3bce-92cf-e86978c295e1,684d5dd7-5964-33ef-81e5-d4070e7b7013,fld,18.0,NaN,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,2600979.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
5,5,812ccf3d-adf8-369c-954a-5beae39db404,684d5dd7-5964-33ef-81e5-d4070e7b7013,fld,13.0,NaN,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,590599.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
6,6,cf401f5b-dc6b-3336-96d9-5ba4bbe049a7,36ab1e06-94ef-363f-841c-0e923bfbdac7,fld,10.0,NaN,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,693634.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
7,7,f98aa8e6-51b7-35f0-b529-0048d4581bf3,36ab1e06-94ef-363f-841c-0e923bfbdac7,fld,14.0,NaN,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,664097.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
8,8,bbe3a638-eebb-380f-9d17-585f6ff37040,aeef8e65-6b2c-3c7a-aeba-3f594b9255df,fld,35.0,NaN,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,4847746.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
9,9,96049783-b9b5-3650-b215-21d4ee5a39d4,aeef8e65-6b2c-3c7a-aeba-3f594b9255df,fld,14.0,NaN,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,664097.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# see list of all columns
df.columns

Index(['Unnamed: 0', 'section_uuid', 'course_offering_uuid', 'section_type',
       'section_number', 'room_uuid', 'schedule_uuid', 'instructor_id',
       'facility_code', 'room_code', 'start_time', 'end_time', 'mon', 'tues',
       'wed', 'thurs', 'fri', 'sat', 'sun', 'subject_code', 'subject_name',
       'subject_abbreviation', 'course_uuid', 'term_code',
       'course_offering_name', 'course_name', 'course_number',
       'course_and_section', 'a_count', 'ab_count', 'b_count', 'bc_count',
       'c_count', 'd_count', 'f_count', 's_count', 'u_count', 'cr_count',
       'n_count', 'p_count', 'i_count', 'nw_count', 'nr_count', 'other_count'],
      dtype='object')

In [ ]:
# drop 'Unnamed: 0' (repeat of the index)
df.drop(columns = 'Unnamed: 0',
        inplace = True)

In [ ]:
# check
df.columns

Index(['section_uuid', 'course_offering_uuid', 'section_type',
       'section_number', 'room_uuid', 'schedule_uuid', 'instructor_id',
       'facility_code', 'room_code', 'start_time', 'end_time', 'mon', 'tues',
       'wed', 'thurs', 'fri', 'sat', 'sun', 'subject_code', 'subject_name',
       'subject_abbreviation', 'course_uuid', 'term_code',
       'course_offering_name', 'course_name', 'course_number',
       'course_and_section', 'a_count', 'ab_count', 'b_count', 'bc_count',
       'c_count', 'd_count', 'f_count', 's_count', 'u_count', 'cr_count',
       'n_count', 'p_count', 'i_count', 'nw_count', 'nr_count', 'other_count'],
      dtype='object')

## Grades

Our target variables will be 'a_proportion', 'f_proportion', and 'avg_grade'. We will calculate and create these with our existing data and then drop the other grade counts columns.

Information from the University of Wisconsin's website about grades can be found here: https://guide.wisc.edu/undergraduate/#enrollmentandrecordstext

Grades:
- a, ab, b, bc, c, d, f - letter grades
  - ab - intermediate grade between a and b
  - bc - intermediate grade between b and c
- s - satisfactory, used in pass/fail courses
- u - unsatisfactory, used in pass/fail courses
- cr - credit, used in credit/no credit courses
- n - no credit, used in credit/no credit courses
- p - progress: temporary grade used for courses extending beyond one term
- i - incomplete: temporary grade used when work is not completed during a term
- nw - no work: for students who enroll in a course and then never attend
- nr - no report: a grade was not submitted by the instructor
- other - any one of several other grading codes

For a preliminary machine learning attempt with this data, we are interested in predicting any one of the following:
1. the number of As as a proportion of total letter grades
2. the number of Fs as a proportion of total letter grades
3. the average letter grade 

To keep the problem simple, we will only count letter grades given in each course. Although the dataset provides counts for some other grade types (S, U, N, CR, etc.), we are interested in the letter grades given (A, AB, B, BC, C, D, and F). So, we will drop the columns representing other grade types. 

To calculate this number, we will change our dataframe (grades_df) to end up with the following columns:
- course_offering_uuid (unchanged)
- section_number (unchanged)
- a_proportion (number of a grades / number of total grades given per row)
- f_proportion (number of f grades / number of total grades given per row)
- average_grade ((sum of letters * points) / number of total grades given per row); points as follows:
  - a: 4.0
  - ab: 3.5
  - b: 3.0
  - bc: 2.5
  - c: 2.0
  - d: 1.0
  - f: 0.0

In [ ]:
# drop following grade counts columns (we are only interested in letter grades at this point):
df.drop(columns = ['s_count', 'u_count', 'cr_count', 'n_count', 'p_count', 'i_count', 'nw_count', 'nr_count', 'other_count'], inplace = True)

In [ ]:
# for every row in grades_df:
for i in range(len(df)):

  # assign variable num_a to the 'a_count' value
  num_a = df.at[i, 'a_count']

  # assign variable num_f to the 'f_count' value
  num_f = df.at[i, 'f_count']

  # assign variable num_all_grades to the sum of all letter grade counts
  num_a = df.at[i, 'a_count']
  num_ab = df.at[i, 'ab_count']
  num_b = df.at[i, 'b_count']
  num_bc = df.at[i, 'bc_count']
  num_c = df.at[i, 'c_count']
  num_d = df.at[i, 'd_count']
  num_f = df.at[i, 'f_count']

  num_all_grades = num_a + num_ab + num_b + num_bc + num_c + num_d + num_f

  # create column for num_all_grades to easily be able to delete courses where no grades were recorded
  df.at[i, 'num_all_grades'] = num_all_grades

  # if num_all_grades is not 0 (we want to exclude rows where there are no recorded grades)
  if num_all_grades != 0:

    # calculate the a_proportion & assign to grades_df
    a_proportion = (num_a) / (num_all_grades)
    df.at[i, 'a_proportion'] = a_proportion

    # calculate the f_proportion & assign to grades_df
    f_proportion = (num_f) / (num_all_grades)
    df.at[i, 'f_proportion'] = f_proportion

    # calculate the average grade & assign to grades_df
    avg_grade = ((num_a * 4.0) + (num_ab * 3.5) + (num_b * 3.0) + (num_bc * 2.5) + (num_c * 2.0) + (num_d * 1.0)) / num_all_grades
    df.at[i, 'avg_grade'] = avg_grade


In [ ]:
# how to do above action without a for loop?

# this way doesn't prevent divide-by-zero error if there are no grades recorded
# put in if statement?
'''
grades_df['a_proportion] = grades_df['a_count'] / \
                            (grades_df['a_count'] +
                             grades_df['ab_count'] +
                             grades_df['b_count'] +
                             grades_df['bc_count'] +
                             grades_df['c_count'] +
                             grades_df['d_count'] +
                             grades_df['f_count']) if (grades_df['a_count'] +
                                                        grades_df['ab_count'] +
                                                        grades_df['b_count'] +
                                                        grades_df['bc_count'] +
                                                        grades_df['c_count'] +
                                                        grades_df['d_count'] +
                                                        grades_df['f_count']) != 0
'''
'''
grades_df['a_proportion'] = grades_df['a_count'] / \
                            (grades_df['a_count'] +
                             grades_df['ab_count'] +
                             grades_df['b_count'] +
                             grades_df['bc_count'] +
                             grades_df['c_count'] +
                             grades_df['d_count'] +
                             grades_df['f_count'])

grades_df['f_proportion'] = grades_df['f_count'] / \
                            (grades_df['a_count'] +
                             grades_df['ab_count'] +
                             grades_df['b_count'] +
                             grades_df['bc_count'] +
                             grades_df['c_count'] +
                             grades_df['d_count'] +
                             grades_df['f_count'])

grades_df['avg_grade'] = ( (grades_df['a_count'] * 4.0) +
                           (grades_df['ab_count'] * 3.5) +
                           (grades_df['b_count'] * 3.0) +
                           (grades_df['bc_count'] * 2.5) +
                           (grades_df['c_count'] * 2.0) +
                           (grades_df['d_count'] * 1.0) +
                           (grades_df['f_count'] * 0.0) ) / \
                         (  grades_df['a_count'] +
                            grades_df['ab_count'] +
                            grades_df['b_count'] +
                            grades_df['bc_count'] +
                            grades_df['c_count'] +
                            grades_df['d_count'] +
                            grades_df['f_count'] )
'''

"\ngrades_df['a_proportion'] = grades_df['a_count'] /                             (grades_df['a_count'] +\n                             grades_df['ab_count'] +\n                             grades_df['b_count'] +\n                             grades_df['bc_count'] +\n                             grades_df['c_count'] +\n                             grades_df['d_count'] +\n                             grades_df['f_count'])\n\ngrades_df['f_proportion'] = grades_df['f_count'] /                             (grades_df['a_count'] +\n                             grades_df['ab_count'] +\n                             grades_df['b_count'] +\n                             grades_df['bc_count'] +\n                             grades_df['c_count'] +\n                             grades_df['d_count'] +\n                             grades_df['f_count'])\n\ngrades_df['avg_grade'] = ( (grades_df['a_count'] * 4.0) +\n                           (grades_df['ab_count'] * 3.5) +\n                           (

In [ ]:
# check
df.columns

Index(['section_uuid', 'course_offering_uuid', 'section_type',
       'section_number', 'room_uuid', 'schedule_uuid', 'instructor_id',
       'facility_code', 'room_code', 'start_time', 'end_time', 'mon', 'tues',
       'wed', 'thurs', 'fri', 'sat', 'sun', 'subject_code', 'subject_name',
       'subject_abbreviation', 'course_uuid', 'term_code',
       'course_offering_name', 'course_name', 'course_number',
       'course_and_section', 'a_count', 'ab_count', 'b_count', 'bc_count',
       'c_count', 'd_count', 'f_count', 'num_all_grades', 'a_proportion',
       'f_proportion', 'avg_grade'],
      dtype='object')

In [ ]:
# check
df[['a_proportion', 'f_proportion', 'avg_grade']].sample(10)

,a_proportion,f_proportion,avg_grade
42118,NaN,NaN,NaN
128076,0.062500,0.00000,3.343750
109815,0.142857,0.02381,2.773810
123527,0.523810,0.00000,3.595238
134753,0.846154,0.00000,3.923077
11573,NaN,NaN,NaN
128477,NaN,NaN,NaN
87916,NaN,NaN,NaN
56373,NaN,NaN,NaN
159222,0.933333,0.00000,3.966667


In [ ]:
# drop rest of grades columns
df.drop(columns = ['a_count', 'ab_count', 'b_count', 'bc_count', 'c_count', 
                   'd_count', 'f_count', 'num_all_grades'],
        inplace = True)

In [ ]:
# check
df.columns

Index(['section_uuid', 'course_offering_uuid', 'section_type',
       'section_number', 'room_uuid', 'schedule_uuid', 'instructor_id',
       'facility_code', 'room_code', 'start_time', 'end_time', 'mon', 'tues',
       'wed', 'thurs', 'fri', 'sat', 'sun', 'subject_code', 'subject_name',
       'subject_abbreviation', 'course_uuid', 'term_code',
       'course_offering_name', 'course_name', 'course_number',
       'course_and_section', 'a_proportion', 'f_proportion', 'avg_grade'],
      dtype='object')

In [ ]:
# delete rows that don't have any values for any of the three target
# variable columns
df.dropna(subset = ['a_proportion', 'f_proportion', 'avg_grade'], 
          how = 'all', 
          inplace = True)

In [ ]:
# check
df.shape

(64186, 30)

## 'term_code'

The current 'term_code' column gives a 4-digit code indicating the term the course was offered in. 

'term_code':
- first digit always 1 (21st century)
- second two digits academic year (from 07 to 18; year that the academic year ends in summer)
- fourth digit term (2 = fall, 4 = spring, 6 = summer)

Source: https://kb.wisc.edu/registrar/117776

In [ ]:
df.reset_index(inplace = True, drop = True)

In [ ]:
# inspect
df['term_code'].value_counts(dropna = False)
# data includes years '07' to '18' and only fall (2) and spring (4) terms

1072.0    3141
1082.0    3055
1092.0    3047
1102.0    3042
1074.0    3006
1152.0    3001
1122.0    2966
1132.0    2946
1112.0    2939
1142.0    2925
1084.0    2921
1162.0    2901
1172.0    2891
1104.0    2869
1094.0    2864
1134.0    2845
1182.0    2832
1114.0    2828
1144.0    2816
1154.0    2810
1164.0    2776
1174.0    2765
Name: term_code, dtype: int64

We will split 'term_code' into one column for year (numeric, 7-18) and one for term (nominal, fall or spring)

In [ ]:
# iterate through all rows in df
for i in range(len(df)):

  # create string of 'term_code' for indexing
  string = df.at[i, 'term_code'].astype(str)

  # make sure string is 4 or more characters long
  if len(string) >= 4:

    # assign 'year' as the middle two characters in the string
    df.at[i, 'year'] = string[1:3]

    # if string ends in 2
    if string[3] == '2':

      # assign 'term' as 'fall'
      df.at[i, 'term'] = 'fall'
  
    # else
    else:

      # assign 'term' as 'spring'
      df.at[i, 'term'] = 'spring'

In [ ]:
# how to do above actions without a for loop?
# this doesn't work

# df['year'] = df['term_code'].astype(str)[1:3]
# df['term'] = df['term_code'].astype(str)[3]

In [ ]:
# check
print(df['year'].value_counts(dropna = False))
print(df['term'].value_counts(dropna = False))

07    6147
08    5976
10    5911
09    5911
15    5811
13    5791
11    5767
14    5741
16    5677
17    5656
12    2966
18    2832
Name: year, dtype: int64
fall      35686
spring    28500
Name: term, dtype: int64


In [ ]:
# drop 'term_code' as it is now redundant
df.drop(columns = 'term_code', inplace = True)

In [ ]:
# check
df.columns

Index(['section_uuid', 'course_offering_uuid', 'section_type',
       'section_number', 'room_uuid', 'schedule_uuid', 'instructor_id',
       'facility_code', 'room_code', 'start_time', 'end_time', 'mon', 'tues',
       'wed', 'thurs', 'fri', 'sat', 'sun', 'subject_code', 'subject_name',
       'subject_abbreviation', 'course_uuid', 'course_offering_name',
       'course_name', 'course_number', 'course_and_section', 'a_proportion',
       'f_proportion', 'avg_grade', 'year', 'term'],
      dtype='object')

## Rooms and buildings

Every room_uuid has a facility_code which indicates a building, and a room_code which indicates a room in the building. We will drop the room_code column because it is likely a high-cardinality variable, since there are many, many rooms on campus. We will keep the building (facility_code). 

In [ ]:
df.drop(columns = ['room_uuid', 'room_code'], inplace = True)

In [ ]:
# check
df.columns

Index(['section_uuid', 'course_offering_uuid', 'section_type',
       'section_number', 'schedule_uuid', 'instructor_id', 'facility_code',
       'start_time', 'end_time', 'mon', 'tues', 'wed', 'thurs', 'fri', 'sat',
       'sun', 'subject_code', 'subject_name', 'subject_abbreviation',
       'course_uuid', 'course_offering_name', 'course_name', 'course_number',
       'course_and_section', 'a_proportion', 'f_proportion', 'avg_grade',
       'year', 'term'],
      dtype='object')

## Schedules

'start_time' and 'end_time' will probably have a high degree of co-linearity due to many classes being on a fixed schedule. So, we can drop 'end_time'. 

But, we are interested in if the length of a class may explain some variance in the grades earned, so first we will calculate 'class_length'. 'class_length' will represent the length of the class in minutes on any day of the week that the class meets (if class meets from 9:00am to 9:50am on MWF, 'class_length' will be 50 minutes)). 

We are also interested in if the total contact time (per week) may explain some variance. So we will calculate 'total_time', which will be the 'class_length' multiplied by the number of classes per week (so if class meets from 9:00am to 9:50am on MWF, 'total_time' will be 150 minutes). 

In [ ]:
# 'class_length'
df['class_length'] = df['end_time'] - df['start_time']

In [ ]:
df.reset_index(inplace = True, drop = True)

In [ ]:
# 'total_time'
df['total_time'] = df['class_length'] * (df['mon'] + df['tues'] + df['wed'] + df['thurs'] + df['fri'] + df['sat'] + df['sun'])

In [ ]:
# check
df[['total_time', 'class_length']]

,total_time,class_length
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
...,...,...
64181,50.0,50.0
64182,50.0,50.0
64183,50.0,50.0
64184,50.0,50.0


Check value_counts for days of the week

In [ ]:
for day in ['mon', 'tues', 'wed', 'thurs', 'fri', 'sat', 'sun']:
  print(df[day].value_counts())

False    36546
True     27640
Name: mon, dtype: int64
False    36954
True     27232
Name: tues, dtype: int64
False    37056
True     27130
Name: wed, dtype: int64
False    38275
True     25911
Name: thurs, dtype: int64
False    48729
True     15457
Name: fri, dtype: int64
False    62543
True      1643
Name: sat, dtype: int64
False    64121
True        65
Name: sun, dtype: int64


Combine classes that meet on weekends into one category to increase number of counts.

In [ ]:
# how to make this section of code fewer lines (with an or statement?)

for i in range(len(df)):
  if df.at[i, 'sat'] == True:
    df.at[i, 'weekend'] = True
  elif df.at[i, 'sun'] == True:
    df.at[i, 'weekend'] = True
  else:
    df.at[i, 'weekend'] = False

In [ ]:
'''
# how to do above without a for loop?

if df['sat'] == True:
  df['weekend'] = True
elif df['sun'] == True:
  df['weekend'] = True
else:
  df['weekend'] = False
'''

"\n# how to do above without a for loop?\n\nif df['sat'] == True:\n  df['weekend'] = True\nelif df['sun'] == True:\n  df['weekend'] = True\nelse:\n  df['weekend'] = False\n"

In [ ]:
# check
df['weekend'].value_counts()

False    62503
True      1683
Name: weekend, dtype: int64

In [ ]:
# delete 'sat' and 'sun' classes (highly correlated with 'weekend'), and
# 'end_time' (highly correlated with 'start_time' and 'class_length')
df.drop(columns = ['sat', 'sun', 'end_time'], inplace = True)

In [ ]:
# check
df.columns

Index(['section_uuid', 'course_offering_uuid', 'section_type',
       'section_number', 'schedule_uuid', 'instructor_id', 'facility_code',
       'start_time', 'mon', 'tues', 'wed', 'thurs', 'fri', 'subject_code',
       'subject_name', 'subject_abbreviation', 'course_uuid',
       'course_offering_name', 'course_name', 'course_number',
       'course_and_section', 'a_proportion', 'f_proportion', 'avg_grade',
       'year', 'term', 'class_length', 'total_time', 'weekend'],
      dtype='object')

In [ ]:
# drop 'schedule_uuid'; it correlates with the other schedule variables
df.drop(columns = 'schedule_uuid', inplace = True)

In [ ]:
# check
df.columns

Index(['section_uuid', 'course_offering_uuid', 'section_type',
       'section_number', 'instructor_id', 'facility_code', 'start_time', 'mon',
       'tues', 'wed', 'thurs', 'fri', 'subject_code', 'subject_name',
       'subject_abbreviation', 'course_uuid', 'course_offering_name',
       'course_name', 'course_number', 'course_and_section', 'a_proportion',
       'f_proportion', 'avg_grade', 'year', 'term', 'class_length',
       'total_time', 'weekend'],
      dtype='object')

## Subjects

Delete 'subject_abbreviations' and 'subject_codes', as these are repeats of the 'subject_name' column.

In [ ]:
df.drop(columns = ['subject_code', 'subject_abbreviation'], inplace = True)

In [ ]:
# check
df.columns

Index(['section_uuid', 'course_offering_uuid', 'section_type',
       'section_number', 'instructor_id', 'facility_code', 'start_time', 'mon',
       'tues', 'wed', 'thurs', 'fri', 'subject_name', 'course_uuid',
       'course_offering_name', 'course_name', 'course_number',
       'course_and_section', 'a_proportion', 'f_proportion', 'avg_grade',
       'year', 'term', 'class_length', 'total_time', 'weekend'],
      dtype='object')

## Course number

In [ ]:
df['course_number'].describe()

count    64186.000000
mean       446.496713
std        258.355618
min          1.000000
25%        223.000000
50%        400.000000
75%        676.000000
max        999.000000
Name: course_number, dtype: float64

The 'course_number' column includes numbers in the following ranges:
- below 100: below college-level
- 100-299: elementary (undergraduate students only)
- 300-499: intermediate (grad & undergrad)
- 500-699: advanced (grad & undergrad)
- 700+: graduate students only

Source: https://grad.wisc.edu/documents/course-numbering-system/

We will split up the courses into these ranges to reduce the cardinality of the variable.

We will drop the below college-level and graduate student only courses and retain just courses that are elementary, intermediate, or advanced, and that undergraduate students may take. (But we will wait until after merging with big_df to do so, so that we delete the other information for these courses as well.)

In [ ]:
for i in range(len(df)):
  if df.at[i, 'course_number'] < 100:
    df.at[i, 'course_difficulty'] = 'below level'
  elif df.at[i, 'course_number'] < 300:
    df.at[i, 'course_difficulty'] = 'elementary'
  elif df.at[i, 'course_number'] < 500:
    df.at[i, 'course_difficulty'] = 'intermediate'
  elif df.at[i, 'course_number'] < 700:
    df.at[i, 'course_difficulty'] = 'advanced'
  elif df.at[i, 'course_number'] >= 700:
    df.at[i, 'course_difficulty'] = 'grad level'

In [ ]:
# how to do above action without a for loop?
# courses_df['course_difficulty'] = 

In [ ]:
# check
df['course_difficulty'].value_counts()

elementary      19562
intermediate    18574
grad level      14130
advanced        11355
below level       565
Name: course_difficulty, dtype: int64

In [ ]:
# we are only interested in 'elementary', 'intermediate', and 'advanced' courses
df.drop(df[(df['course_difficulty'] == 'below level') | 
           (df['course_difficulty'] == 'grad level')].index, 
        inplace = True)


In [ ]:
# check
df['course_difficulty'].value_counts()

elementary      19562
intermediate    18574
advanced        11355
Name: course_difficulty, dtype: int64

In [ ]:
# drop course_number column
df.drop(columns = 'course_number', inplace = True)

In [ ]:
# check
df.columns

Index(['section_uuid', 'course_offering_uuid', 'section_type',
       'section_number', 'instructor_id', 'facility_code', 'start_time', 'mon',
       'tues', 'wed', 'thurs', 'fri', 'subject_name', 'course_uuid',
       'course_offering_name', 'course_name', 'course_and_section',
       'a_proportion', 'f_proportion', 'avg_grade', 'year', 'term',
       'class_length', 'total_time', 'weekend', 'course_difficulty'],
      dtype='object')

## course_uuid

In [ ]:
# can drop 'course_uuid' column because it should match up with 'course_name' column
df.drop(columns = 'course_uuid', inplace = True)

In [ ]:
# check
df.columns

Index(['section_uuid', 'course_offering_uuid', 'section_type',
       'section_number', 'instructor_id', 'facility_code', 'start_time', 'mon',
       'tues', 'wed', 'thurs', 'fri', 'subject_name', 'course_offering_name',
       'course_name', 'course_and_section', 'a_proportion', 'f_proportion',
       'avg_grade', 'year', 'term', 'class_length', 'total_time', 'weekend',
       'course_difficulty'],
      dtype='object')

### Reduce cardinality in instructor_id (binning):
- Group instructors who have taught less than 20 sections into one group.

In [ ]:
print(len(df['instructor_id']))
print(len(df))

49491
49491


In [ ]:
print(df['instructor_id'].value_counts())

566960.0     132
496397.0     106
2601912.0     89
2601706.0     85
319547.0      81
            ... 
4716883.0      1
4721762.0      1
3375280.0      1
3457484.0      1
900462.0       1
Name: instructor_id, Length: 7106, dtype: int64


In [ ]:
# create empty list to store instructor_ids
instructor_20_list = []

# iterate over values and counts in 'instructor_id'
for value, count in df['instructor_id'].value_counts().items():
  
  # if count is more than 20 (i.e. instructor taught more than 20 sections)
  if count > 20:
    
    # append to list
    instructor_20_list.append(value)

# this is the number of instructor_ids who taught more than 20 sections
print(len(instructor_20_list))
print(type(instructor_20_list))
print(instructor_20_list[:20])

521
<class 'list'>
[566960.0, 496397.0, 2601912.0, 2601706.0, 319547.0, 157057.0, 663146.0, 783847.0, 2601527.0, 623858.0, 4232086.0, 309711.0, 464468.0, 1005245.0, 980056.0, 3013497.0, 412406.0, 3615604.0, 3216300.0, 2601502.0]


In [ ]:
# assign new column where 'instructor_id' is copied if instructor taught more
# than 20 sections, and where 'other' if not

df.loc[~df['instructor_id'].isin(instructor_20_list), 'instructor_id'] = 'other'

In [ ]:
# check
df['instructor_id'].sample(10)

58869     550736.0
5179         other
18420    4124270.0
3862         other
34674        other
3929         other
30850        other
34785        other
42743        other
39615        other
Name: instructor_id, dtype: object

In [ ]:
# how many unique values now in 'instructor_id_250?
df['instructor_id'].nunique()

# this can be played with later to see if higher cardinality will help the model

522

### Reduce cardinality in course_name (binning):

In [ ]:
# how many unique values are in 'course_name'?
df['course_name'].nunique()

3771

In [ ]:
# look at value_counts
df['course_name'].value_counts()

introduction to speech composition                                               997
special topics                                                                   705
freshman composition                                                             652
third semester spanish                                                           463
intermediate language practice with emphasis on writing and grammar              459
                                                                                ... 
contemporary skill set literature in french                                        1
elementary student teaching seminar - elementary/special education dual major      1
seminar in electrical and computer engineering                                     1
the history of evolutionary thought                                                1
soccer ii                                                                          1
Name: course_name, Length: 3771, dtype: int64

In [ ]:
# create empty list to store course_names
course_name_20_list = []

# iterate over values and counts in 'course_name'
for value, count in df['course_name'].value_counts().items():
  
  # if count is more than 20 (i.e. course has been taught more than 50 times)
  if count > 20:
    
    # append to list
    course_name_20_list.append(value)

# this is the number of courses that have been taught more than 20 times
print(len(course_name_20_list))

521


In [ ]:
# assign new column where 'course_name' is copied if course was taught more
# than 20 times, and where 'other' if not
df.loc[~df['course_name'].isin(course_name_20_list), 'course_name'] = 'other'

In [ ]:
# check
df['course_name'].sample(10)

20504                 introduction to hispanic literatures
21084    human resources development: educational effec...
48108                                                other
38355                                                other
43182                          advanced seminar in history
41893                                                other
14738                  introduction to college composition
16784                                                other
5101                              advanced ensemble-string
17005                                              algebra
Name: course_name, dtype: object

In [ ]:
# how many unique values now in 'course_name'?
df['course_name'].nunique()

# this can be played with later to see if higher cardinality will help the model

522

### Drop course_offering_name and course_offering_uuid

In [ ]:
# course_offering_name may be correlated with course_name, so we will drop it for now
# course_offering_uuid will not predict grades, so we will drop it for now
df.drop(columns = ['course_offering_name', 'course_offering_uuid'], inplace = True)

In [ ]:
# check
df.columns

Index(['section_uuid', 'section_type', 'section_number', 'instructor_id',
       'facility_code', 'start_time', 'mon', 'tues', 'wed', 'thurs', 'fri',
       'subject_name', 'course_name', 'course_and_section', 'a_proportion',
       'f_proportion', 'avg_grade', 'year', 'term', 'class_length',
       'total_time', 'weekend', 'course_difficulty'],
      dtype='object')

### section_uuid, section_number

In [ ]:
#drop section_uuid, section_number, and grades_section_number because
# they will not be predictive of grades
df.drop(columns = ['section_uuid', 'section_number'], inplace = True)

In [ ]:
# check
df.columns

Index(['section_type', 'instructor_id', 'facility_code', 'start_time', 'mon',
       'tues', 'wed', 'thurs', 'fri', 'subject_name', 'course_name',
       'course_and_section', 'a_proportion', 'f_proportion', 'avg_grade',
       'year', 'term', 'class_length', 'total_time', 'weekend',
       'course_difficulty'],
      dtype='object')

## Final checks

In [ ]:
# check unique values and value_counts()
for col in df.columns:
  print(f"{col}:")
  print(f"unique values: {df[col].nunique()}")
  print(f"value counts: \n{df[col].value_counts(dropna = False)}")
  print()

section_type:
unique values: 6
value counts: 
lec    39207
lab     4810
sem     2570
ind     2113
fld      549
dis      242
Name: section_type, dtype: int64

instructor_id:
unique values: 522
value counts: 
other        32772
566960.0       132
496397.0       106
2601912.0       89
2601706.0       85
             ...  
537957.0        21
475025.0        21
367808.0        21
172570.0        21
4131298.0       21
Name: instructor_id, Length: 522, dtype: int64

facility_code:
unique values: 119
value counts: 
0482     7447
0469     4561
NaN      3837
0140     3446
0408     2532
         ... 
0039        1
0084b       1
0033        1
1400g       1
1400e       1
Name: facility_code, Length: 120, dtype: int64

start_time:
unique values: 118
value counts: 
 660.0     7599
 800.0     4392
 595.0     4315
-1.0       3976
 870.0     3628
           ... 
 925.0        1
 665.0        1
 1055.0       1
 1040.0       1
 550.0        1
Name: start_time, Length: 118, dtype: int64

mon:
unique values

In [ ]:
# course_and_section is uniquely identifying each row, so it can be dropped
df.drop(columns = 'course_and_section', inplace = True)

In [ ]:
# check cardinality
for col in df.columns:
  print(col)
  print(df[col].nunique())
  print()

# except for the target variables, all of the columns have a number of unique values less than 450

section_type
6

instructor_id
522

facility_code
119

start_time
118

mon
2

tues
2

wed
2

thurs
2

fri
2

subject_name
180

course_name
522

a_proportion
4137

f_proportion
1308

avg_grade
6955

year
12

term
2

class_length
61

total_time
61

weekend
2

course_difficulty
3



In [ ]:
df.reset_index(drop = True)

,section_type,instructor_id,facility_code,start_time,mon,tues,wed,thurs,fri,subject_name,course_name,a_proportion,f_proportion,avg_grade,year,term,class_length,total_time,weekend,course_difficulty
0,lec,other,NaN,-1.0,False,False,False,False,False,communication arts,introduction to speech composition,0.250000,0.000000,3.625000,15,spring,0.0,0.0,False,elementary
1,lec,other,0545,660.0,False,True,False,True,False,communication arts,introduction to speech composition,0.230769,0.000000,3.346154,15,spring,75.0,75.0,False,elementary
2,lec,other,0545,660.0,False,True,False,True,False,communication arts,introduction to speech composition,0.153846,0.000000,3.500000,15,spring,75.0,75.0,False,elementary
3,lec,other,0545,595.0,True,False,True,False,True,communication arts,introduction to speech composition,0.500000,0.000000,3.750000,15,spring,50.0,50.0,False,elementary
4,lec,other,0545,595.0,True,False,True,False,True,communication arts,introduction to speech composition,0.363636,0.000000,3.454545,15,spring,50.0,50.0,False,elementary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49486,lec,other,0093,800.0,True,True,False,False,True,comparative biosciences,other,0.111111,0.012346,3.000000,09,fall,50.0,50.0,False,advanced
49487,lec,other,0093,800.0,True,True,False,False,True,comparative biosciences,other,0.209877,0.000000,3.308642,11,fall,50.0,50.0,False,advanced
49488,lec,other,0093,800.0,True,True,False,False,True,comparative biosciences,other,0.261364,0.000000,3.210227,16,fall,50.0,50.0,False,advanced
49489,lec,other,0093,800.0,True,True,False,False,True,comparative biosciences,other,0.225000,0.012500,3.087500,10,fall,50.0,50.0,False,advanced


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49491 entries, 12 to 64185
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   section_type       49491 non-null  object 
 1   instructor_id      49491 non-null  object 
 2   facility_code      45654 non-null  object 
 3   start_time         49491 non-null  float64
 4   mon                49491 non-null  bool   
 5   tues               49491 non-null  bool   
 6   wed                49491 non-null  bool   
 7   thurs              49491 non-null  bool   
 8   fri                49491 non-null  bool   
 9   subject_name       49491 non-null  object 
 10  course_name        49491 non-null  object 
 11  a_proportion       49491 non-null  float64
 12  f_proportion       49491 non-null  float64
 13  avg_grade          49491 non-null  float64
 14  year               49491 non-null  object 
 15  term               49491 non-null  object 
 16  class_length       49

In [ ]:
df.head(10)

,section_type,instructor_id,facility_code,start_time,mon,tues,wed,thurs,fri,subject_name,course_name,a_proportion,f_proportion,avg_grade,year,term,class_length,total_time,weekend,course_difficulty
12,lec,other,NaN,-1.0,False,False,False,False,False,communication arts,introduction to speech composition,0.250000,0.0,3.625000,15,spring,0.0,0.0,False,elementary
13,lec,other,0545,660.0,False,True,False,True,False,communication arts,introduction to speech composition,0.230769,0.0,3.346154,15,spring,75.0,75.0,False,elementary
14,lec,other,0545,660.0,False,True,False,True,False,communication arts,introduction to speech composition,0.153846,0.0,3.500000,15,spring,75.0,75.0,False,elementary
15,lec,other,0545,595.0,True,False,True,False,True,communication arts,introduction to speech composition,0.500000,0.0,3.750000,15,spring,50.0,50.0,False,elementary
16,lec,other,0545,595.0,True,False,True,False,True,communication arts,introduction to speech composition,0.363636,0.0,3.454545,15,spring,50.0,50.0,False,elementary
17,lec,other,0545,595.0,True,False,True,False,True,communication arts,introduction to speech composition,0.333333,0.0,3.416667,15,spring,50.0,50.0,False,elementary
18,lec,other,0545,595.0,True,False,True,False,True,communication arts,introduction to speech composition,0.400000,0.0,3.550000,15,spring,50.0,50.0,False,elementary
19,lec,other,0545,660.0,True,False,True,False,True,communication arts,introduction to speech composition,0.384615,0.0,3.653846,15,spring,50.0,50.0,False,elementary
20,lec,other,0545,660.0,True,False,True,False,True,communication arts,introduction to speech composition,0.083333,0.0,3.291667,15,spring,50.0,50.0,False,elementary
21,lec,other,0545,660.0,True,False,True,False,True,communication arts,introduction to speech composition,0.333333,0.0,3.333333,15,spring,50.0,50.0,False,elementary


## Export to csv

In [ ]:
df.to_csv('/content/drive/MyDrive/Coding Dojo/Project 2/all_grades_data_cleaned.csv')

## Further refinement:

There are catergories here that can potentially be grouped up into larger categories; keep this in mind to come back to and test the algorithm on (e.g. asian languages and cultures and asian languages and cultures: languages). 

Consider some way to reduce co-linearity among days of the week columns (there is probably a pattern that many classes meet mwf or tr, and we want to minimize the effect of this pattern). 

# Final data dictionary

Feature columns:
- section_type: such as lecture, discussion, field, etc.
- instructor_id: instructor
- facility_code: building
- start_time: start time of class in minutes (-1: no start time assigned)
- mon, tues, wed, thurs, fri, weekend: if class meets on that day
- subject_name: subject
- year: academic year (calendar year of spring term)
- term: fall or spring
- course_name: same for same course across terms
- course_difficulty: based on course numbering system
- class_length: number of minutes between start and end time
- total_time: number of contact minutes per week

Target columns:
- a_proportion, f_proportion: proportion (0-1) of A/F letter grades awarded (out of all letter grades awarded)
- avg_grade: average grade on 4.0 scale counting all letter grades awarded